In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split


In [2]:
from vehicle_detection.sliding_window import single_img_features
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        current_features = single_img_features(image, cspace, spatial_size=(32,32),hist_bins=32,
                                              orient=orient, pix_per_cell=pix_per_cell,
                                              cell_per_block=cell_per_block,
                                              hog_channel=hog_channel)
        # Append the new feature vector to the features list
        #print("l:", len(current_features))
        features.append(current_features)
    # Return list of feature vectors
    print("Extracted {} features.".format(len(features)))
    return features



In [3]:
# Divide up into cars and notcars
images = glob.glob('*.jpeg')
cars = glob.glob('../vehicles/**/*.png')
notcars = glob.glob('../non-vehicles/**/*.png')
print("Found {} cars and {} non-cars.".format(len(cars), len(notcars)))

Found 8792 cars and 8968 non-cars.


In [4]:
# Reduce the sample size because HOG features are slow to compute
# The quiz evaluator times out after 13s of CPU time
#sample_size = 500
#cars = cars[0:sample_size]
#notcars = notcars[0:sample_size]

In [5]:
cs = cars[0:3]
print(cs)
f = extract_features(cs, 'HSV', 9, 8, 2, 'ALL')

['../vehicles/GTI_Far/image0000.png', '../vehicles/GTI_Far/image0001.png', '../vehicles/GTI_Far/image0002.png']
Extracted 3 features.


In [6]:

def train_and_evaluate_svm(colorspace = 'RGB', orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0):
    t=time.time()
    car_features = extract_features(cars, cspace=colorspace, orient=orient, 
                            pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                            hog_channel=hog_channel)
    notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                            pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                            hog_channel=hog_channel)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to extract HOG features...')
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64) 
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.2, random_state=rand_state)

    print('Using:',orient,'orientations',pix_per_cell,
        'pixels per cell and', cell_per_block,'cells per block',
        'in color space', colorspace,'and hog channel',hog_channel)
    print('Feature vector length:', len(X_train[0]))
    # Use a linear SVC 
    svc = LinearSVC()
    # Check the training time for the SVC
    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC...')
    # Check the score of the SVC
    print("="*30)
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    print("="*30)
    # Check the prediction time for a single sample
    if False:
        t=time.time()
        n_predict = 15
        print('My SVC predicts:     ', svc.predict(X_test[0:n_predict]))
        print('For these',n_predict, 'labels: ', y_test[0:n_predict])
        t2 = time.time()
        print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
    del car_features
    del notcar_features
    del X
    del scaled_X
    return svc, X_scaler
#train_and_evaluate_svm()

In [7]:
# quick and good:
# cell_per_block: 3, colorspace: HSV, hog_channel: ALL, orient: 6, pix_per_cell: 12, 
#p = {'cell_per_block': 2, 'colorspace': 'HSV', 'orient': 16, 'pix_per_cell': 8, 'hog_channel': 'ALL'}
#my_svm, my_scaler = train_and_evaluate_svm('HSV', 6, 12, 3, 'ALL')
#p = {'pix_per_cell': 8, 'hog_channel': 'ALL', 'orient': 8, 'cell_per_block': 3, 'colorspace': 'HSV'}
p =  {'pix_per_cell': 10, 'colorspace': 'YUV', 'cell_per_block': 2, 'orient': 9, 'hog_channel': 0}
my_svm, my_scaler = train_and_evaluate_svm(colorspace=p['colorspace'], 
                                           orient=p['orient'], 
                                           pix_per_cell=p['pix_per_cell'], 
                                           cell_per_block=p['cell_per_block'], 
                                           hog_channel=p['hog_channel'])


Extracted 8792 features.
Extracted 8968 features.
28.29 Seconds to extract HOG features...
Using: 9 orientations 10 pixels per cell and 2 cells per block in color space YUV and hog channel 0
Feature vector length: 900
6.37 Seconds to train SVC...
Test Accuracy of SVC =  0.9524


In [8]:
import pickle

with open('hog_svm.p', 'wb') as f:
    pickle.dump((p, my_svm, my_scaler), f)

In [ ]:
# My gridsearch

class Choose:            
    def __init__(self, what):
        self.what = what
    def sample(self):
        idx = np.random.randint(0, len(self.what))
        return self.what[idx]

def sample_all(params):
    all_params = {}
    for key in params.keys():
        all_params[key] = params[key].sample()
    return all_params
    
    
def search(n, function, params):
    for i in range(n):
        p = sample_all(params)
        print("p = ", p)
        function(colorspace = p['colorspace'], orient=p['orient'], 
                 pix_per_cell=p['pix_per_cell'], 
                 cell_per_block=p['cell_per_block'], 
                 hog_channel=p['hog_channel'])
        



In [ ]:
params = {
    "colorspace": Choose(['RGB', 'HSV', 'HLS', 'YUV', 'YCrCb']),
    "orient": Choose([8, 9, 10, 12]),
    "pix_per_cell": Choose([6,8,10]),
    "cell_per_block": Choose([2,3,4]),
    "hog_channel": Choose([0, 1, 2, 'ALL']),
}

np.random.seed(42)

search(100, train_and_evaluate_svm, params)

p =  {'cell_per_block': 4, 'hog_channel': 'ALL', 'colorspace': 'HLS', 'pix_per_cell': 10, 'orient': 8}
Extracted 8792 features.
Extracted 8968 features.
69.65 Seconds to extract HOG features...
Using: 8 orientations 10 pixels per cell and 4 cells per block in color space HLS and hog channel ALL
Feature vector length: 3456
6.31 Seconds to train SVC...
Test Accuracy of SVC =  0.9761
p =  {'cell_per_block': 2, 'hog_channel': 2, 'colorspace': 'HLS', 'pix_per_cell': 10, 'orient': 9}
Extracted 8792 features.
Extracted 8968 features.
28.46 Seconds to extract HOG features...
Using: 9 orientations 10 pixels per cell and 2 cells per block in color space HLS and hog channel 2
Feature vector length: 900
12.46 Seconds to train SVC...
Test Accuracy of SVC =  0.9091
p =  {'cell_per_block': 2, 'hog_channel': 'ALL', 'colorspace': 'YCrCb', 'pix_per_cell': 10, 'orient': 12}
Extracted 8792 features.
Extracted 8968 features.
96.55 Seconds to extract HOG features...
Using: 12 orientations 10 pixels per cell

In [ ]:
params = {
    "colorspace": Choose(['RGB', 'HSV', 'HLS']),
    "orient": Choose([8, 9, 10, 12, 14, 16]),
    "pix_per_cell": Choose([6,8,10]),
    "cell_per_block": Choose([2,3,4]),
    "hog_channel": Choose([0, 1, 2, 'ALL']),
}


np.random.seed(42)

search(100, train_and_evaluate_svm, params)

In [ ]:

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), normalise=True, 
                                  visualise=vis)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), normalise=True, 
                       visualise=vis)
        return features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features
